In [ ]:
import os
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from simpleeval import simple_eval
from typing import Dict
from langchain_core.tools import tool


os.environ["GOOGLE_API_KEY"] = ""

llm = init_chat_model("google_genai:gemini-2.0-flash")

AgentState = Dict[str, str]

def chatbot(state:AgentState)->AgentState:
    return {"message": [llm.invoke(state["message"])]}
graph = StateGraph(AgentState)



@tool
def mycalc(state: AgentState) -> AgentState:
    """
    A simple calculator tool that evaluates basic math expressions using BODMAS rules.
    Input: state["message"] should contain a math expression like '3 + 4 * 2'
    Output: state["message"] will be updated with the result as a string.
    """
    try:
        allowed = "0123456789/*+-(). "
        expression = state["message"]

        if not all(char in allowed for char in expression):
            return {"message": "Invalid characters in expression."}
        
        result = simple_eval(expression)

        return {"message": str(result)} 
    except Exception as e:
        return {"message": f"Error: {str(e)}"}
    
llm_with_tools = llm.bind_tools([mycalc])



In [94]:
def chatbot(state:AgentState)->AgentState:
    return {"message":{llm_with_tools.invoke(state["message"])}}

In [95]:
graph.add_node("chatbot",chatbot)

In [98]:
graph.add_edge(START,"chatbot")